# Aim of Notebook:
To get insights using Google Analytics to show **what type of information we can gather** (bearing in mind Munich RE presentation).

# Loading the Data:

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
import os
import glob
import xlrd
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2 import service_account
from google.cloud import bigquery
from apiclient.discovery import build
import pygsheets

ModuleNotFoundError: No module named 'google.cloud'

In [2]:
os.chdir('/Volumes/GoogleDrive/My Drive/SC_DataAnalysis')

In [3]:
#google sheets sales data
key_file = 'Data/Credentials/BigQuery_Credentials/fast-nexus-262509-41d3a850166f.json'
sheet_key = '10dKsjIUXMg3Pux2-Gd_v-1RAZ24lE5zjMMv_wVprkLU'

In [4]:
#for google analytics
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
key_file = 'Data/Credentials/BigQuery_Credentials/fast-nexus-262509-41d3a850166f.json'
KEY_FILE_LOCATION = key_file
VIEW_ID = '184447081'

In [5]:
#for bigquery funnel views
credentials = service_account.Credentials.from_service_account_file('Data/Credentials/BigQuery_Credentials/funnel-singular-cover-352171e974cc.json')
project_id = 'funnel-singular-cover'
client = bigquery.Client(credentials=credentials, project=project_id)

In [6]:
#categories file for homogenization
categories_file = 'Data/categories.xlsx'

In [7]:
def load_gsheet(cred_file, sheet_id, sheet_name):
    client = pygsheets.authorize(service_account_file=cred_file)
    g_sheet = client.open_by_key(sheet_id)
    sheet = g_sheet.worksheet_by_title(sheet_name)
    df = sheet.get_as_df()
    for i, j in enumerate(df[df.columns[0]]):
        if j == '' and df.iloc[i+1][df.columns[0]] == '':
            df = df.iloc[:i,:].reset_index(drop=True)
            break
    return df

In [8]:
sales = load_gsheet(key_file, sheet_key, 'Ventas')

In [9]:
sales

,Fecha Evento,Nombre,Sub-categoria,Email,Phone,Billing Cycle,Policy Number,Policy URL,Current Price,Policy Price,...,Match email,Match telefono,Semana,Mes,WP,Dia Lead,Días,Mes lead,Año,Objetivos Venta
0,01/02/2019,"PALACIOS SADABA, REMEDIOS",panaderia,magu1959@yahoo.es,637568637,anual,45107286,,301.31,301.31,...,,Sin asignar,5,2,€301.31,08/12,None,None,None,None
1,12/02/2019,"LOPEZ LOPEZ, MARIA CARMEN",centro estetica,carmen.estetic@gmail.com,600882906,anual,45111396,,374.69,374.69,...,Sin asignar,Sin asignar,7,2,€374.69,N/A,None,None,None,None
2,12/02/2019,"REFOJOS GONZALEZ, IVAN",estudio fotografia,info@photoestudio.es,698100788,anual,45111756,,176.46,176.46,...,Sin asignar,Sin asignar,7,2,€176.46,N/A,None,None,None,None
3,12/02/2019,INVERSIONES AULAGA S.L.,adm. fincas,acanto16@yahoo.es,610556677,anual,45118483,,192.47,207.86,...,Sin asignar,Sin asignar,7,2,€207.86,N/A,,,,3
4,13/02/2019,"SAN JOSE LOPEZ, ABRAHAM",pub,naps100@gmail.com,615273583,anual,45114444,,620.33,620.33,...,,Sin asignar,7,2,€620.33,14/03,,,,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2663,17/11/2020,Adalberto Gomes,Restaurante,harusushi1@hotmail.com,623160699,Mensual,04CM27116089,,8.79 €,140.75 €,...,adwords,Sin asignar,47,11,€140.75,17/11,0,11,2020,None
2664,23/11/2020,Luis Paillacho Males,Electricista,alangio_1604@hotmail.com,640506401,Mensual,04RC57027814,,13.26 €,159.22 €,...,adwords,Sin asignar,48,11,€159.22,23/11,0,11,2020,None
2665,24/11/2020,Destileria Maria La Cruz SC,Destileria,alvaromatagrande@gmail.com,629771392,Mensual,48024615,,46.49 €,557.90 €,...,Sin asignar,Sin asignar,48,11,€557.90,N/A,,#VALUE!,#VALUE!,None
2666,24/11/2020,María Belén Abellan Megias,Estilista,bel_90@live.com,669989839,Mensual,04CM27116294,,11.71 €,140.48 €,...,adwords,Sin asignar,48,11,€140.48,19/11,5,11,2020,None


In [10]:
def analytics_query(view_id, index, start_date, end_date, dimensions, metrics, key_file):
    credentials = credentials = service_account.Credentials.from_service_account_file(key_file)
    VIEW_ID = '184447081'
    service = build('analytics', 'v3', credentials=credentials)
    data = service.data().ga().get(
        ids='ga:' + view_id,
        start_date=start_date,
        end_date=end_date,
        metrics=metrics,
        dimensions=dimensions,
        start_index=index,
        max_results=10000).execute()
    
    return data

In [11]:
def initialize_analyticsreporting():
    credentials = ServiceAccountCredentials.from_json_keyfile_name(
      KEY_FILE_LOCATION, SCOPES)
    analytics = build('analyticsreporting', 'v4', credentials=credentials)
    return analytics

def analytics_query(view_id, index, start_date, end_date, dimensions, metrics, key_file):
    credentials = credentials = service_account.Credentials.from_service_account_file(key_file)
    VIEW_ID = '184447081'
    service = build('analytics', 'v3', credentials=credentials)
    data = service.data().ga().get(
        ids='ga:' + view_id,
        start_date=start_date,
        end_date=end_date,
        metrics=metrics,
        dimensions=dimensions,
        start_index=index,
        max_results=10000).execute()
    
    return data

def main_analytics(start_date, end_date, dimensions, metrics, key_file):    
    analytics = initialize_analyticsreporting()
    dimensions = 'ga:'+(',ga:'.join(dimensions))
    metrics = 'ga:'+(',ga:'.join(metrics))
    
    return analytics_query('184447081', 1, start_date, end_date, dimensions, metrics, key_file)

In [12]:
def query_pd(variables, project, group, table):
    query_job = client.query(f"""
    SELECT {variables} FROM {project}.{group}.{table};""")
    
    return query_job.to_dataframe()

def read_data(table_list):
    df_list = []
    for table in table_list:
        df_list.append(query_pd('*', 'funnel-singular-cover', 'funnel_views', f'{table}_filtered'))
    
    return df_list

def read_sales_data(key_file, sheet_key):
    sales = load_gsheet(key_file, sheet_key, 'Ventas')
    sales = sales.rename(columns = {'Fecha Evento': 'Sales Date', 'Email': 'email'})
    sales = sales.drop(['Nombre', 'Phone', 'Días', 'Mes lead', 'Año', 'Objetivos Venta'], axis = 1)
    sales['Sales Date'] = sales['Sales Date'].apply(lambda x: x[-4:] + '-' + x[3:5] + '-' + x[:2])
    
    return sales

def merge_data1(df_list, sales_df): #selecting all data using email and Date to merge
    merge_df = df_list[0]
    for i, df_ in enumerate(df_list):
        if i == 1:
            df_['Type'] = 'Comercio'
        columns1 = set(merge_df.columns)
        columns2 = set(df_.columns)
        intersect_col = list(columns1.intersection(columns2))
        intersect_col.remove('email')
        intersect_col.remove('Date')
        df_ = df_.drop(intersect_col, axis =1)
        merge_df = merge_df.merge(df_, on = ['email', 'Date'], how = 'left')
    if 'phone' in merge_df.columns:
        merge_df = merge_df.drop(['phone'], axis =1)
    
    rows = []
    rows.append(merge_df.shape[0])
    merge_df = merge_df.merge(sales_df, on = 'email', how = 'left')
    rows.append(merge_df.shape[0])
    if rows[0] != rows[1]:
        print("There are duplicates in the merge with sales, please check.")
        print("pre-merger rows:", rows[0], "post-merger rows:", rows[1])
    
    merge_df.reset_index(drop=True, inplace=True)
    
    return merge_df
    
def merge_data2(df_list, sales_df): #select only where google analytics userId is existing to merge
    merge_df = df_list[0]
    merge_df = merge_df.loc[~merge_df['userId'].isnull()]
    for i, df_ in enumerate(df_list):
        if i == 1:
            df_['Type'] = 'Comercio'
        df_ = df_.loc[~df_['userId'].isnull()]
        columns1 = set(merge_df.columns)
        columns2 = set(df_.columns)
        intersect_col = list(columns1.intersection(columns2))
        intersect_col.remove('userId')
        intersect_col.remove('Date')
        df_ = df_.drop(intersect_col, axis =1)
        merge_df = merge_df.merge(df_, on = ['userId', 'Date'], how = 'left')
    if 'phone' in merge_df.columns:
        merge_df = merge_df.drop(['phone'], axis =1)
    
    rows = []
    rows.append(merge_df.shape[0])
    merge_df = merge_df.merge(sales_df, on = 'email', how = 'left')
    rows.append(merge_df.shape[0])
    if rows[0] != rows[1]:
        print("There are duplicates in the merge with sales, please check.")
        print("pre-merger rows:", rows[0], "post-merger rows:", rows[1])
    
    merge_df.reset_index(drop=True, inplace=True)
    
    return merge_df

def process_data(df):
    df_final = df.copy()
    df_final['quote'] = df_final['premiumPrice1'].apply(lambda x: 0 if str(x).lower() == 'nan' else 1)
    df_final['sale'] = df_final['Sales Date'].apply(lambda x: 0 if str(x).lower() == 'nan' else 1)
    df_final = df_final.loc[df_final.email.str.contains('test') == False]
    df_final = df_final.loc[df_final.email.str.contains('singularcover') == False]
    df_final.reset_index(inplace=True, drop=True)
    
    return df_final

def add_categories(df_final, categories_file):
    categories = pd.read_excel(categories_file)
    type_dict = {}
    for cat in df_final['Sub_Categoria']:
        if cat not in categories['extCode'].unique():
            type_dict[cat] = np.nan
        else:
            if categories.loc[categories['extCode'] == cat]['Comercio'].values[0] == 1:
                type_dict[cat] = 'Comercio'
            if categories.loc[categories['extCode'] == cat]['RC'].values[0] == 1:
                type_dict[cat] = 'RC'
    
    for cat in df_final.loc[df_final['Type'] == 'Comercio'].Sub_Categoria.unique():
        if cat in type_dict.keys():
            type_dict[cat] = 'Comercio'
    
    for key, value in type_dict.items():
        df_final.loc[df_final['Sub_Categoria'] == key, 'Type'] = value
        
    extcode_categoria = {}
    for i, j in zip(categories['extCode'].to_list(), categories['Categoria'].to_list()):
        extcode_categoria[i] = j
        
    categoria_conv = {}
    for i in df_final['Sub_Categoria'].unique():
        if i in extcode_categoria.keys():
            categoria_conv[i] = extcode_categoria[i]
        else:
             categoria_conv[i] = np.nan
                
    for i in categoria_conv.keys():
        df_final.loc[df_final['Sub_Categoria'] == i, 'Categoria'] = categoria_conv[i]
        
    return df_final

def convert_timestamp(x):
    if str(x).lower() == 'nan' or len(str(x)) < 6:
        return x
    else:
        x = str(x)[:19]
        x = x.replace('T', ' ')
        x = x.replace('Z', '')
        x = datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
        return x

def update_timestamp_values(df, timestamp_list):
    for i in range(1, len(timestamp_list)):
        df[timestamp_list[i-1]] = df[timestamp_list[i-1]].apply(lambda x: convert_timestamp(x))
        df[timestamp_list[i]] = df[timestamp_list[i]].apply(lambda x: convert_timestamp(x))
        start_name = timestamp_list[i-1][-2:]; end_name = timestamp_list[i][-2:]
        df[f'time_diff_{start_name}_{end_name}'] = df[timestamp_list[i]] - df[timestamp_list[i-1]]
    return df

In [13]:
def main():
    sales_df = read_sales_data(key_file, sheet_key)
    df_list = [about_you, activity, bienes, offers, fiscal_data, payment]
    merge_df = merge_data1(df_list, sales_df)
    df_final = process_data(merge_df)
    df_final = add_categories(df_final, categories_file)
    timestamp_list = ['eventTime_ay', 'eventTime_ac', 'eventTime_vg', 'eventTime_off',
                      'eventTime_fd', 'eventTime_pm']
    df_final = update_timestamp_values(df_final, timestamp_list)
        
    return df_final

In [14]:
def main_to_df(returned_data):
    device_df = pd.DataFrame(returned_data['rows'], columns = [x['name'][3:] for x in returned_data['columnHeaders']])
    device_df['date'] = device_df['date'].apply(lambda x: x[:4] + '-' + x[4:6] + '-' + x[6:])
    device_df = device_df.rename(columns = {'date': 'Date', 'dimension2': 'userId'})
    return device_df

def device_geo(df, dimensions_list, metrics_list, key_file):
    min_date = df.Date.min(); absolute_max = df.Date.max()
    df_list = []
    device = main_analytics(min_date, absolute_max, dimensions_list, metrics_list, key_file)
    device_df = main_to_df(device)
    df_list.append(device_df)
    while df_list[-1].Date.max() < absolute_max:
        min_date = df_list[-1].Date.max()
        print(min_date)
        device_new = main_analytics(min_date, absolute_max, dimensions_list, metrics_list, key_file)
        device_df_new = main_to_df(device_new)
        df_list.append(device_df_new)
    else:
        df_new = df_list[0]
        for df_ in df_list:
            df_new = pd.concat([df_new, df_], axis = 0)
        df_new = df_new.drop_duplicates()
    return df_new

In [15]:
%%time
table_list = ['about_you', 'activity', 'valuation_goods', 'offers', 'fiscal_data', 'payment']
about_you, activity, bienes, offers, fiscal_data, payment = read_data(table_list)

CPU times: user 489 ms, sys: 209 ms, total: 698 ms
Wall time: 52.9 s


In [16]:
%%time
data = main()

There are duplicates in the merge with sales, please check.
pre-merger rows: 21178 post-merger rows: 21495
CPU times: user 14 s, sys: 80.8 ms, total: 14.1 s
Wall time: 16 s


In [17]:
data_ = data.loc[~data.userId.isnull()].reset_index(drop=True)

In [18]:
dimensions = ['date', 'dimension2', 'operatingSystem', 'deviceCategory', 'city', 'latitude', 'longitude']
metrics = ['sessions']3

SyntaxError: invalid syntax (<ipython-input-18-7023bdb12c25>, line 2)

In [ ]:
%%time
device_df1 = device_geo(data_, dimensions, metrics, key_file)

In [ ]:
dimensions = ['date', 'dimension2', 'city', 'cityId', 'region', 'regionId']
metrics = ['sessions']

In [ ]:
%%time
device_df2 = device_geo(data_, dimensions, metrics, key_file)

In [ ]:
df_final = data_.merge(device_df1, on = ['userId', 'Date'], how = 'left')
df_final = data_.merge(device_df2, on = ['userId', 'Date'], how = 'left').reset_index(drop=True)

In [ ]:
df_final.shape

In [ ]:
pd.set_option("display.max_columns", 150)
df_final.head()

In [ ]:
df_final.to_csv('data.csv', index=False)

# Munich RE: What can we show?
Behavioural information regarding types of users we have.

In [40]:
df_final['city'] = df_final['city'].apply(lambda x: np.nan if x == '(not set)' else x)
df_final['cityId'] = df_final['cityId'].apply(lambda x: np.nan if x == '(not set)' else x)

In [41]:
df_final['city'].isnull().sum()

1871

In [42]:
df_final.loc[df_final.sale == 1].userId.nunique()

702

In [43]:
sales = load_gsheet(key_file, sheet_key, 'Ventas')

In [44]:
sales.shape

(2529, 32)

However, referencing this time period; we have 1143 sales; therefore there is an issue here. That is, we are not picking up enough of the sales.

The reason for this is that a lot of these sales will have a different email address associated with them OR that these sales simply are not within 

**Therefore, when showcasing a geographical analysis to Munich RE we can differentiate: between sales on sales alone data (using therefore the data compiled in Ventas sheet for this analysis) AND/OR the merged data with funnel related data.**

## Integrating Demographic Data:

In [45]:
def get_series(codigo, num_datos):
    url_plantilla = f'http://servicios.ine.es/wstempus/js/EN/DATOS_SERIE/{codigo}?nult={num_datos}'
    resp = requests.get(url_plantilla)
    datos = resp.json()
    
    values_dict = {}
    values_dict[codigo] = datos['Data'][0]['Valor']
        
    return values_dict
    
def get_comunidad_data(url, population=False, salario=False):
    re = requests.get(url)
    re = re.json()

    cod_dict = {}
    for i, dict_ in enumerate(re):
        if population == True:
            if 'Todas las edades' not in dict_['Nombre']:
                re = re[:i]
                break
        if salario == True:
            if 'Mujeres' in dict_['Nombre']:
                re = re[:i]
                break
            if 'Hombres' in dict_['Nombre']:
                re = re[:i]
                break
        if population == True:
            cod_dict[dict_['COD']] = dict_['Nombre'].split('.')[:3]
        if salario == True:
            cod_dict[dict_['COD']] = dict_['Nombre'].split('.')[:-1]
            
    data = []
    for i, cod in enumerate(cod_dict.keys()):
        data.append(get_series(cod, 1))
    
    return data, cod_dict

In [46]:
def convert_to_df_pop(data_, cod_info_):
    codes = []
    values = []
    for i in data_:
        for key, value in i.items():
            codes.append(key)
            values.append(value)
    region = []
    sex = []
    for key, value in cod_info_.items():
        region.append(value[1])
        sex.append(value[2])
    df = pd.DataFrame()
    df['codes'] = codes
    df['region'] = region
    df['sex'] = sex
    df['values'] = values
    
    return df

def convert_to_df_salario(data_, cod_info_):
    codes = []
    values = []
    for i in data_:
        for key, value in i.items():
            codes.append(key)
            values.append(value)
    region = []
    salario = []
    for key, value in cod_info_.items():
        region.append(value[1])
        salario.append(value[-1])
    df = pd.DataFrame()
    df['codes'] = codes
    df['region'] = region
    df['salario'] = salario
    df['values'] = values
    
    return df

In [47]:
import requests, datetime, json
url = "https://servicios.ine.es/wstempus/js/ES/SERIES_TABLA/9681?page=1"
data, cod_info = get_comunidad_data(url, population = True)
df_pop = convert_to_df_pop(data, cod_info)

In [48]:
url = "https://servicios.ine.es/wstempus/js/ES/SERIES_TABLA/28191?page=1"
data, cod_info = get_comunidad_data(url, salario = True)
df_salario = convert_to_df_salario(data, cod_info)

In [49]:
df_pop.head()

,codes,region,sex,values
0,CP335,Todas las edades,Total,47329981.0
1,CP300335,Todas las edades,Hombres,23197625.0
2,CP300334,Todas las edades,Mujeres,24132356.0
3,CP390551,Andalucía,Total,8476718.0
4,CP390550,Andalucía,Hombres,4182541.0


In [50]:
df_salario.head()

,codes,region,salario,values
0,EAES741,Total Nacional,Media,24009.12
1,EAES740,Total Nacional,10,8457.36
2,EAES739,Total Nacional,25,13998.29
3,EAES738,Total Nacional,50,20078.44
4,EAES737,Total Nacional,75,30057.33


In [51]:
def rename_region(df):
    df.loc[df['region'] == ' Andalucía', 'region'] = 'Andalusia'
    df.loc[df['region'] == ' Aragón', 'region'] = 'Aragon'
    df.loc[df['region'] == ' Asturias, Principado de', 'region'] = 'Asturias'
    df.loc[df['region'] == ' Balears, Illes', 'region'] = 'Balearic Islands'
    df.loc[df['region'] == ' Canarias', 'region'] = 'Canary Islands'
    df.loc[df['region'] == ' Cantabria', 'region'] = 'Cantabria'
    df.loc[df['region'] == ' Castilla y León', 'region'] = 'Castile and Leon'
    df.loc[df['region'] == ' Castilla - La Mancha', 'region'] = 'Castile-La Mancha'
    df.loc[df['region'] == ' Cataluña', 'region'] = 'Catalonia'
    df.loc[df['region'] == ' Comunitat Valenciana', 'region'] = 'Valencian Community'
    df.loc[df['region'] == ' Extremadura', 'region'] = 'Extremadura'
    df.loc[df['region'] == ' Galicia', 'region'] = 'Galicia'
    df.loc[df['region'] == ' Madrid, Comunidad de', 'region'] = 'Community of Madrid'
    df.loc[df['region'] == ' Murcia, Región de', 'region'] = 'Murcia'
    df.loc[df['region'] == ' Navarra, Comunidad Foral de', 'region'] = 'Navarre'
    df.loc[df['region'] == ' País Vasco', 'region'] = 'Basque Country'
    df.loc[df['region'] == ' Rioja, La', 'region'] = 'La Rioja'
    return df

In [52]:
df_pop_clean = rename_region(df_pop)
df_salario_clean = rename_region(df_salario)

In [53]:
df_pop_clean = df_pop.loc[df_pop_clean['sex'] == ' Total']
df_salario_clean = df_salario_clean.loc[df_salario_clean['salario'] == ' Media']

In [54]:
base = df_final.copy()

In [55]:
df_final = df_final.merge(df_pop_clean[['region', 'values']], on = 'region', how = 'left')

In [56]:
df_final = df_final.rename(columns = {'values': 'population'})

In [57]:
df_final = df_final.merge(df_salario_clean[['region', 'values']],
                          on = 'region', how = 'left')

In [58]:
df_final = df_final.rename(columns = {'values': 'region_mean_income'})

In [59]:
df_final

,eventTime_ay,Date,email,firstName,lastName,Sub_Categoria,campaign,medium,userId,row_numbers,...,time_diff_vg_ff,time_diff_ff_fd,time_diff_fd_pm,city,cityId,region,regionId,sessions,population,region_mean_income
0,2020-03-19 11:57:29,2020-03-19,eduardcalzadasanchez@gmail.com,eduard,calzada,director-de-agencia-de-viajes,None,None,194f1cf1-2e9c-4088-97e3-38fa3124742f,1,...,0 days 00:00:01,NaT,NaT,NaN,NaN,Catalonia,20278,1,7652069.0,25552.84
1,2020-03-19 15:05:27,2020-03-19,kamanspain@gmail.com,Antonio,Callejón Martín,operador-de-maquinaria-de-construccion,01_España_Genéricas,ppc,59515876-4ccf-460c-a98a-f56427d580c4,1,...,0 days 00:00:03,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-03-19 16:51:13,2020-03-19,beltranlastrap@gmail.com,Gabriel,Mendoza Exposito,oficina,None,None,687198a9-69e7-414f-8f90-232dadeef78f,1,...,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-03-19 17:03:07,2020-03-19,blancabecquer11@gmai.com,Laura,Sachez,minimarket,ES - Gen - Seguro Locales / local comercial / ...,ppc,4f7454f3-09c2-4b32-b48d-bad620cdd8f7,1,...,0 days 00:00:03,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-03-19 17:14:56,2020-03-19,vem_desarrollo@gmail.com,Victor Emmanuel,Molinas,oficina,ES - Gen - Seguro Locales / local comercial / ...,ppc,21f03e02-1bf7-4018-99bb-7264cb804540,1,...,NaT,NaT,NaT,NaN,NaN,Balearic Islands,21387,10,1210750.0,22800.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7125,2020-11-03 13:35:13,2020-11-03,usoalmenara@gmail.com,Natalia,Alme,maestro-de-educacion-infantil,10430315940,cpc,7f2bfbd8-1cdf-4ea9-bb42-f689e544a89c,1,...,NaT,NaT,NaT,Madrid,1005493,Madrid,9047045,1,NaN,NaN
7126,2020-11-03 13:51:15,2020-11-03,construccionesgalera@hotmail.com,JOSE MANUEL,GALERA CRESPO,encargado-de-la-construccion,10430285424,cpc,fb8d1c71-7e62-4bf2-befd-c2b64a2414b1,1,...,0 days 00:00:03,NaT,NaT,NaN,NaN,Aragon,21386,3,1330445.0,23531.83
7127,2020-11-03 14:04:46,2020-11-03,b02f61105s@iesarucasdomingorivero.com,mireia,almeida,tienda-ropa,10430316405,cpc,1146ff5f-0bbb-4386-bba1-fecf8b168fcc,1,...,NaT,NaT,NaT,NaN,NaN,Canary Islands,20277,3,2237309.0,20763.48
7128,2020-11-03 14:18:38,2020-11-03,ars.10oct@gmail.com,Alessandro,Rocca,casa-rural,10430316405,cpc,061d9639-df86-465b-8f2a-8f943adca634,1,...,0 days 00:02:22,NaT,NaT,NaN,NaN,La Rioja,20281,5,315926.0,22641.17


# Correlation Analysis: Looking at Geography

In [60]:
df_final.isnull().sum

<bound method DataFrame.sum of       eventTime_ay   Date  email  firstName  lastName  Sub_Categoria  \
0            False  False  False      False     False          False   
1            False  False  False      False     False          False   
2            False  False  False      False     False          False   
3            False  False  False      False     False          False   
4            False  False  False      False     False          False   
...            ...    ...    ...        ...       ...            ...   
7125         False  False  False      False     False          False   
7126         False  False  False      False     False          False   
7127         False  False  False      False     False          False   
7128         False  False  False      False     False          False   
7129         False  False  False      False     False          False   

      campaign  medium  userId  row_numbers  ...  time_diff_vg_ff  \
0         True    True   False     

In [61]:
df_final.columns

Index(['eventTime_ay', 'Date', 'email', 'firstName', 'lastName',
       'Sub_Categoria', 'campaign', 'medium', 'userId', 'row_numbers',
       'eventTime_ac', 'gclid', 'country', 'state', 'town', 'zipcode',
       'policy_start_date', 'policy_end_date', 'floors', 'goods', 'surface',
       'inauguration_year', 'locationRisk', 'numLocals', 'shop_postcalcode',
       'refurbishing_year', 'shop_region', 'electronic_protections',
       'fire_protections', 'theft_protections', 'watchman_protections', 'Type',
       'eventTime_vg', 'gross_income', 'money_invested',
       'furnitureWithEquipment', 'merchandise', 'ownership', 'eventTime_off',
       'monthlypayment1', 'premiumPrice1', 'monthlypayment2', 'premiumPrice2',
       'monthlypayment3', 'premiumPrice3', 'provider', 'eventTime_fd',
       'birthdate', 'documenttype', 'offerchosen', 'billingcycle',
       'eventTime_pm', 'subcription_id', 'intent_status', 'endDate',
       'paymenttype', 'policynumber', 'policy_url', 'policyprice',
  

In [62]:
select_columns = ['Date', 'email', 'Sub_Categoria', 'userId', 'state', 'town', 'zipcode', 'floors', 'surface',
                 'inauguration_year', 'locationRisk', 'refurbishing_year', 'shop_region', 'gross_income', 
                 'money_invested', 'furnitureWithEquipment', 'merchandise', 'ownership', 'region', 'city', 
                 'population', 'region_mean_income']

In [63]:
df1 = df_final[select_columns]

In [64]:
df1.shape

(7130, 22)

In [65]:
df1.isnull().sum()

Date                         0
email                        0
Sub_Categoria                0
userId                       0
state                     3896
town                      3896
zipcode                   3896
floors                    3896
surface                   3896
inauguration_year         3896
locationRisk              3896
refurbishing_year         6610
shop_region               3896
gross_income              3052
money_invested            5200
furnitureWithEquipment    4727
merchandise               4728
ownership                 4727
region                     517
city                      1871
population                2183
region_mean_income        2183
dtype: int64

In [66]:
pd.options.mode.chained_assignment = None  # default='warn'
df1['gross_income'] = df1['gross_income'].astype(float)
df1['money_invested'] = df1['money_invested'].astype(float)
df1['furnitureWithEquipment'] = df1['furnitureWithEquipment'].astype(float)
df1['merchandise'] = df1['merchandise'].astype(float)

In [67]:
df1.dtypes

Date                       object
email                      object
Sub_Categoria              object
userId                     object
state                      object
town                       object
zipcode                    object
floors                     object
surface                    object
inauguration_year          object
locationRisk               object
refurbishing_year          object
shop_region                object
gross_income              float64
money_invested            float64
furnitureWithEquipment    float64
merchandise               float64
ownership                  object
region                     object
city                       object
population                float64
region_mean_income        float64
dtype: object

In [68]:
df1.corr()

,gross_income,money_invested,furnitureWithEquipment,merchandise,population,region_mean_income
gross_income,1.000000,0.078719,0.555144,0.162488,0.011130,0.019162
money_invested,0.078719,1.000000,0.115030,0.048330,-0.047258,-0.022519
furnitureWithEquipment,0.555144,0.115030,1.000000,0.180125,0.008569,0.022225
merchandise,0.162488,0.048330,0.180125,1.000000,0.003558,0.008247
population,0.011130,-0.047258,0.008569,0.003558,1.000000,0.148724
region_mean_income,0.019162,-0.022519,0.022225,0.008247,0.148724,1.000000


Okay so cannot see any particular correlation here betwene cases of comercio value and the region where it is.

In [72]:
pd.set_option("display.max_columns", 100)
df1

,Date,email,Sub_Categoria,userId,state,town,zipcode,floors,surface,inauguration_year,locationRisk,refurbishing_year,shop_region,gross_income,money_invested,furnitureWithEquipment,merchandise,ownership,region,city,population,region_mean_income
0,2020-03-19,eduardcalzadasanchez@gmail.com,director-de-agencia-de-viajes,194f1cf1-2e9c-4088-97e3-38fa3124742f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30000.0,NaN,NaN,NaN,None,Catalonia,NaN,7652069.0,25552.84
1,2020-03-19,kamanspain@gmail.com,operador-de-maquinaria-de-construccion,59515876-4ccf-460c-a98a-f56427d580c4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30000.0,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
2,2020-03-19,beltranlastrap@gmail.com,oficina,687198a9-69e7-414f-8f90-232dadeef78f,Andalucía,Sevilla,41013,1,100,1999,URBAN_HUB,None,ES-AN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-03-19,blancabecquer11@gmai.com,minimarket,4f7454f3-09c2-4b32-b48d-bad620cdd8f7,Andalucía,Sevilla,41002,1,90,1993,URBAN_HUB,None,ES-AN,25000.0,7000.0,3000.0,5000.0,TENANT,NaN,NaN,NaN,NaN
4,2020-03-19,vem_desarrollo@gmail.com,oficina,21f03e02-1bf7-4018-99bb-7264cb804540,Principado de Asturias,Avilés,33402,1,70,1975,URBAN_HUB,None,ES-AS,10000.0,NaN,5000.0,3000.0,TENANT,Balearic Islands,NaN,1210750.0,22800.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7125,2020-11-03,usoalmenara@gmail.com,maestro-de-educacion-infantil,7f2bfbd8-1cdf-4ea9-bb42-f689e544a89c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Madrid,Madrid,NaN,NaN
7126,2020-11-03,construccionesgalera@hotmail.com,encargado-de-la-construccion,fb8d1c71-7e62-4bf2-befd-c2b64a2414b1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30000.0,NaN,NaN,NaN,None,Aragon,NaN,1330445.0,23531.83
7127,2020-11-03,b02f61105s@iesarucasdomingorivero.com,tienda-ropa,1146ff5f-0bbb-4386-bba1-fecf8b168fcc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Canary Islands,NaN,2237309.0,20763.48
7128,2020-11-03,ars.10oct@gmail.com,casa-rural,061d9639-df86-465b-8f2a-8f943adca634,País Vasco,Elciego,01340,2,500,2004,URBAN_HUB,None,ES-PV,250000.0,NaN,30000.0,8000.0,TENANT,La Rioja,NaN,315926.0,22641.17


In [73]:
df1

,Date,email,Sub_Categoria,userId,state,town,zipcode,floors,surface,inauguration_year,locationRisk,refurbishing_year,shop_region,gross_income,money_invested,furnitureWithEquipment,merchandise,ownership,region,city,population,region_mean_income
0,2020-03-19,eduardcalzadasanchez@gmail.com,director-de-agencia-de-viajes,194f1cf1-2e9c-4088-97e3-38fa3124742f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30000.0,NaN,NaN,NaN,None,Catalonia,NaN,7652069.0,25552.84
1,2020-03-19,kamanspain@gmail.com,operador-de-maquinaria-de-construccion,59515876-4ccf-460c-a98a-f56427d580c4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30000.0,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
2,2020-03-19,beltranlastrap@gmail.com,oficina,687198a9-69e7-414f-8f90-232dadeef78f,Andalucía,Sevilla,41013,1,100,1999,URBAN_HUB,None,ES-AN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-03-19,blancabecquer11@gmai.com,minimarket,4f7454f3-09c2-4b32-b48d-bad620cdd8f7,Andalucía,Sevilla,41002,1,90,1993,URBAN_HUB,None,ES-AN,25000.0,7000.0,3000.0,5000.0,TENANT,NaN,NaN,NaN,NaN
4,2020-03-19,vem_desarrollo@gmail.com,oficina,21f03e02-1bf7-4018-99bb-7264cb804540,Principado de Asturias,Avilés,33402,1,70,1975,URBAN_HUB,None,ES-AS,10000.0,NaN,5000.0,3000.0,TENANT,Balearic Islands,NaN,1210750.0,22800.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7125,2020-11-03,usoalmenara@gmail.com,maestro-de-educacion-infantil,7f2bfbd8-1cdf-4ea9-bb42-f689e544a89c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Madrid,Madrid,NaN,NaN
7126,2020-11-03,construccionesgalera@hotmail.com,encargado-de-la-construccion,fb8d1c71-7e62-4bf2-befd-c2b64a2414b1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30000.0,NaN,NaN,NaN,None,Aragon,NaN,1330445.0,23531.83
7127,2020-11-03,b02f61105s@iesarucasdomingorivero.com,tienda-ropa,1146ff5f-0bbb-4386-bba1-fecf8b168fcc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Canary Islands,NaN,2237309.0,20763.48
7128,2020-11-03,ars.10oct@gmail.com,casa-rural,061d9639-df86-465b-8f2a-8f943adca634,País Vasco,Elciego,01340,2,500,2004,URBAN_HUB,None,ES-PV,250000.0,NaN,30000.0,8000.0,TENANT,La Rioja,NaN,315926.0,22641.17


# Defining our KPI: What is the last stage they were at before sale?

In [76]:
df_final[['quote', 'sale']].corr()

,quote,sale
quote,1.000000,0.082428
sale,0.082428,1.000000


In [79]:
df_final[['quote', 'sale']].isnull().sum()

quote    0
sale     0
dtype: int64

In [81]:
df_final.shape

(7130, 99)

In [105]:
events = ['eventTime_ay','eventTime_ac','eventTime_vg','eventTime_off','eventTime_fd','eventTime_pm']
latest_stage = {'about_you': [], 'activity': [], 'valuation_goods': [], 'offer': [], 'fiscal_data': [], 'payment': []}

In [106]:
for i in range(df_final.shape[0]):
    for event, stage in zip(events, list(latest_stage.keys())):
        if str(df_final.iloc[i][event]).lower() == 'nat':
            latest_stage[stage].append(0)
        else:
            latest_stage[stage].append(1)

In [112]:
funnel = pd.DataFrame.from_dict(data = latest_stage)

In [113]:
funnel = pd.concat([df_final[['userId', 'sale']], funnel], axis = 1)

In [114]:
funnel = funnel.groupby('userId')[['about_you', 'activity', 'valuation_goods', 'offer', 'fiscal_data', 'payment', 'sale']].sum()

In [115]:
funnel

,about_you,activity,valuation_goods,offer,fiscal_data,payment,sale
userId,,,,,,,
,2,0,2,2,0,0,0
00028659-e5f7-4154-a694-74ca27c36438,1,0,0,0,0,0,1
0005bee8-ad9a-4d99-9c03-2117e9f9ae6e,1,1,1,1,0,0,0
0018f3b4-2f31-4cf4-aabd-a3c94b0ee25f,4,0,4,4,0,0,4
0019143e-353c-4d4a-9576-c8d62dc1ac10,1,0,1,1,1,1,0
...,...,...,...,...,...,...,...
ffdef7d7-72d0-43f3-82a6-104d1ea22427,1,0,1,1,0,0,0
ffe0afc9-1d9f-4a1e-ad51-00a3639f4bc6,1,0,1,1,0,0,0
ffe7a1fe-b69d-4d9f-975a-0a4086538e15,1,1,0,0,0,0,0


In [117]:
df_final.loc[df_final.userId =='0018f3b4-2f31-4cf4-aabd-a3c94b0ee25f']

,eventTime_ay,Date,email,firstName,lastName,Sub_Categoria,campaign,medium,userId,row_numbers,eventTime_ac,gclid,country,state,town,zipcode,policy_start_date,policy_end_date,floors,goods,surface,inauguration_year,locationRisk,numLocals,shop_postcalcode,refurbishing_year,shop_region,electronic_protections,fire_protections,theft_protections,watchman_protections,Type,eventTime_vg,gross_income,money_invested,furnitureWithEquipment,merchandise,ownership,eventTime_off,monthlypayment1,premiumPrice1,monthlypayment2,premiumPrice2,monthlypayment3,premiumPrice3,provider,eventTime_fd,birthdate,documenttype,offerchosen,billingcycle,eventTime_pm,subcription_id,intent_status,endDate,paymenttype,policynumber,policy_url,policyprice,currentprice,Sales Date,Sub-categoria,Billing Cycle,Policy Number,Policy URL,Current Price,Policy Price,Start Date,Payment Type,Persona,Fecha Anulación,Razon anulación,Aseguradora,Online/offline,Renovado,Precio Renovacion,Tipo de póliza,,Match email,Match telefono,Semana,Mes,WP,Dia Lead,quote,sale,Categoria,time_diff_ay_ac,time_diff_ac_vg,time_diff_vg_ff,time_diff_ff_fd,time_diff_fd_pm,city,cityId,region,regionId,sessions,population,region_mean_income
990,2020-05-10 11:02:09,2020-05-10,Angelcortesgarciabarcelona@gmail.com,Ángel,Cortes gracia,servicios-de-limpieza,None,None,0018f3b4-2f31-4cf4-aabd-a3c94b0ee25f,1,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comercio,2020-05-10 11:02:25,30000,None,None,None,None,2020-05-10 11:02:27,1327,15922,2212,26537,2745,32938,liberty-civilLiability,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-11,Limpieza,mensual,47070199,,18.46,221.56,11/05/2020,Tarjeta,Daniel Maroto,,,Allianz,0,,,RC,,,Sin asignar,20.0,5.0,€221.56,10/05,1,1,Servicios generales,NaT,NaT,0 days 00:00:02,NaT,NaT,Barcelona,1005424,Catalonia,20278,1,7652069.0,25552.84
991,2020-05-10 11:02:09,2020-05-10,Angelcortesgarciabarcelona@gmail.com,Ángel,Cortes gracia,servicios-de-limpieza,None,None,0018f3b4-2f31-4cf4-aabd-a3c94b0ee25f,1,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comercio,2020-05-10 11:02:25,30000,None,None,None,None,2020-05-10 11:02:27,1327,15922,2212,26537,2745,32938,liberty-civilLiability,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-11,Limpieza,mensual,47070199,,18.46,221.56,11/05/2020,Tarjeta,Daniel Maroto,,,Allianz,0,,,RC,,,Sin asignar,20.0,5.0,€221.56,10/05,1,1,Servicios generales,NaT,NaT,0 days 00:00:02,NaT,NaT,Sabadell,1005434,Catalonia,20278,2,7652069.0,25552.84
992,2020-05-10 11:02:09,2020-05-10,Angelcortesgarciabarcelona@gmail.com,Ángel,Cortes gracia,servicios-de-limpieza,None,None,0018f3b4-2f31-4cf4-aabd-a3c94b0ee25f,1,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comercio,2020-05-10 11:02:25,30000,None,None,None,None,2020-05-10 11:02:27,1327,15922,2212,26537,2745,32938,liberty-civilLiability,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-11,Convenio,anual,04ACV7174985,,30.12,30.12,11/05/2020,Tarjeta,Daniel Maroto,,,Liberty,0,,,RC,,,Sin asignar,20.0,5.0,€30.12,10/05,1,1,Servicios generales,NaT,NaT,0 days 00:00:02,NaT,NaT,Barcelona,1005424,Catalonia,20278,1,7652069.0,25552.84
993,2020-05-10 11:02:09,2020-05-10,Angelcortesgarciabarcelona@gmail.com,Ángel,Cortes gracia,servicios-de-limpieza,None,None,0018f3b4-2f31-4cf4-aabd-a3c94b0ee25f,1,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comercio,2020-05-10 11:02:25,30000,None,None,None,None,2020-05-10 11:02:27,1327,15922,2212,26537,2745,32938,liberty-civilLiability,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-11,Convenio,anual,04ACV7174985,,30.12,30.12,11/05/2020,Tarjeta,Daniel Maroto,,,Liberty,0,,,RC,,,Sin asignar,20.0,5.0,€30.12,10/05,1,1,Servicios generales,NaT,NaT,0 days 00:00:02,NaT,NaT,Sabadell,1005434,Catalonia,20278,2,7652069.0,25552.84


### Conclusion: need to think of a way that we show this analytics to product clearly. For example: for all of our sales, where are they stopping off from? How many sessions do they do? What is an indicator of a good client purely from their behaviour? 


### Proposal: build a dashboard so that we can identify clearly user behaviour.